# Testing lambert.py 

Built in-house, using Bate's formulation for Lambert Problem

Last updated 2019.11.16

In [1]:
import numpy as np
import logging
import warnings
from numpy.linalg import norm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
from lambert import *


In [2]:
# initial and final position vectors [km]
r1 = np.array([5000, 10000, 2100])
r2 = np.array([-14600, 2500, 7000])

# time of flight [s]
dt = 3600

# gravitational parameter [km^3/s^2]
mu_E = 398600


In [ ]:
# by using the warnings catcher, we can get rid of all the excessive warnings
# infact, if the logging level is set at WARN, none of the info level details are printed either.

logging.basicConfig(level = logging.INFO)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    v1, v2 = lambert(r1=r1, r2=r2, tof=dt, mu=mu_E, grade='pro')

INFO:root:========================= LAMBERT'S PROBLEM =========================
INFO:root:Transfering from r1: [ 5000 10000  2100] [km]
INFO:root:              to r2: [-14600   2500   7000] [km]


In [ ]:
v1, v2

### Try plotting the transfer leg

In [ ]:
elements = sv2el(r1,v1,mu_E)


In [ ]:
def plot_transfer_nb(r1,v1,r2,v2,mu):
    """Function plots transfer orbits resulting from solving Lambert's problem
    Args:
        r1 (lst): initial position vector [km]
        v1 (lst): initial velocity vector [km]
        r2 (lst): final position vector [km/s]
        v2 (lst): final velocity vector [km/s]
        mu (float): gravitational parameter [km^3/s^2]
    Returns:
        (tuple): tupple of position vector and a 3D plot of transfer
    """
    # fetch orbital elements at initial and final positions
    el1 = sv2el(r1,v1,mu)
    el2 = sv2el(r2,v2,mu)

    # define plotting step-size
    n = 200

    # create array of theta
    if el2.get('theta') > el1.get('theta'):
        theta_range = np.linspace(el1.get('theta'),el2.get('theta'),n)
    else:
        theta_range = np.linspace(el2.get('theta'),el1.get('theta'),n)

    # initialize position vector arrays
    r_PF = np.zeros((3,n))
    r_GEC = np.zeros((3,n))
    # compute perifocal and GEC positions
    for i in range(n):
        tmpPF1 = el1.get('h')**2/(mu*(1 + el1.get('e')*np.cos(theta_range[i]))) * np.array([[np.cos(theta_range[i]), np.sin(theta_range[i]), 0]])
        tmpPF1 = np.transpose(tmpPF1)
        tmpPF2 = np.dot(rotMat(-el2.get('omega'),3), tmpPF1)
        tmpPF3 = np.dot(rotMat(-el2.get('i'),1),  tmpPF2)
        tmpGEC = np.dot(rotMat(-el2.get('RAAN'),3), tmpPF3)

        # store computed values
        for j in range(3):
            r_PF[j,i]  = tmpPF1[j]
            r_GEC[j,i] = tmpGEC[j]

    return r_PF, r_GEC

In [ ]:
rPF, rGEC = plot_transfer(r1,v1,r2,v2,mu_E)


In [ ]:
# create PF-frame plot
# plot statistical data of asteroids: inclination vs. semi-major axis
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot()
plt.plot(rPF[0,:], rPF[1,:],'x')
#plt.title(f'Asteroids distribution in {spice.et2utc(et0,"c",0)}')
plt.xlabel('x [km]')
plt.ylabel('y [km]')
#plt.legend()
plt.show()